In [ ]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { BufferMemory } from "langchain/memory";
import { ChatPromptTemplate } from "@langchain/core/prompts";

In [ ]:
const chatModel = new ChatOpenAI({
    verbose:true
});
const memory = new BufferMemory();

const TEMPLATE = `
你是一个乐于助人的 ai 助手。尽你所能回答所有问题。

这是跟人类沟通的聊天历史:
{history}

据此回答人类的问题:
{input}
`
const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

In [ ]:
let tempInput = ""

const chain =  RunnableSequence.from([
    {
        input: new RunnablePassthrough(),
        memoryObject: async (input) => {
            const history = await memory.loadMemoryVariables({
                input
            })
            tempInput = input
            return history
        }
    },
    RunnablePassthrough.assign({
        history: (input) => input.memoryObject.history
    }),
    prompt,
    new RunnablePassthrough({
        func: (input) => console.log(input)
    }),
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (output) => {
            await memory.saveContext({
                input: tempInput
            }, {
                output
            })
        }
    }),
]);

In [ ]:
await chain.invoke("你好, 我叫小明")

In [ ]:
await chain.invoke("我叫什么？")

## 实现自定义的 chat history

In [ ]:
import { JSONChatHistory } from "./JSONChatHistory/index.ts"
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const history = new JSONChatHistory({
    dir: "chat_data",
    sessionId: "test"
})


await history.addMessages([
  new HumanMessage("Hi, 我叫小明"),
  new AIMessage("你好"),
]);

const messages = await history.getMessages();
console.log(messages)

In [ ]:
import { ChatOpenAI } from "@langchain/openai";
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";


const chatModel = new ChatOpenAI();
const memory = new BufferMemory({
    chatHistory: history
});
const chain = new ConversationChain({ llm: chatModel, memory: memory });
const res1 = await chain.call({ input: "我叫什么？" });

In [ ]:
res1

In [ ]:
const messages = await history.getMessages()

In [ ]:
messages